In [4]:
# import the MAAP package to handle queries
from maap.maap import MAAP


# Choose bounding box

The bounding box should be a single string of four lat/lon, in the order:
[left  bottom  right top]

In [6]:
bbox = '-118.06817 34.22169 -118.05801 34.22822'  # Mt Wilson (near JPL) (small, ~8 sec)
# bbox = '-124.81360 32.44506 -113.75989 42.24498' # CA and NV (uses ~5-6 GB RAM, takes a few minutes to run)
# bbox = '-125.53514 25.42150  -87.06960 49.22105' # Western US (CA, WA, Chicago, New Orleans)
# bbox = '-125.41615 31.03621 -101.80916 49.17324' # Western US (CA -> CO)
# bbox =    '6.26868 36.00380   18.57179 47.28139'  # Italy (Requires more than 8 GB RAM)


## Launch jobs in DPS

If you're still in development, skip this section
and jump to "Test Run in ADE" section below.

It's faster to test the algorithm in the ADE first, and
then run it on DPS.

In [3]:
# Register Algorithm from YAML file
# maap = MAAP()
# maap.register_algorithm_from_yaml_file("/projects/GET-DEM-v0.1.0.yml").text

'{"code": 200, "message": {"id": "e24b670f96359f777d4ce1e98ebd3acdd57beb72", "short_id": "e24b670f", "created_at": "2024-01-26T21:15:39.000+00:00", "parent_ids": ["1602b35737e86ad6910ca20eb1c8fd62f6033d50"], "title": "Registering algorithm: GET-DEM-v0.1.0", "message": "Registering algorithm: GET-DEM-v0.1.0", "author_name": "root", "author_email": "root@8e29c9ef876b", "authored_date": "2024-01-26T21:15:39.000+00:00", "committer_name": "root", "committer_email": "root@8e29c9ef876b", "committed_date": "2024-01-26T21:15:39.000+00:00", "trailers": {}, "web_url": "https://repo.maap-project.org/root/register-job-hysds-v4/-/commit/e24b670f96359f777d4ce1e98ebd3acdd57beb72", "stats": {"additions": 0, "deletions": 0, "total": 0}, "status": "created", "project_id": 3, "last_pipeline": {"id": 11683, "iid": 751, "project_id": 3, "sha": "e24b670f96359f777d4ce1e98ebd3acdd57beb72", "ref": "main", "status": "created", "source": "push", "created_at": "2024-01-26T21:15:40.325Z", "updated_at": "2024-01-26T

In [7]:
# Submit DPS Job
maap = MAAP()

jobs = []

for job_num in range(2):
    jobs.append(
        maap.submitJob(
            identifier="test2",  # create a unique job identifier
            algo_id="GET-DEM-v0.1.0",
            version="develop",
            username="niemoell",
            queue="maap-dps-worker-4vcpu-8gb",
            bbox=bbox  # for benchmarking consistency, use same bbox for all jobs
        )
    )


In [19]:
# Monitor the job status via JN.
# For a GUI, go to "View and Submit Jobs" via the Launcher
n_failed = 0
n_succeeded = 0
n_running = 0
n_except = 0
n_accepted = 0

for j in jobs:
    try:
        j.retrieve_attributes()
    except:
        n_except += 1
        continue
    if j.status.lower() == 'failed':
        n_failed += 1
    elif j.status.lower() == 'succeeded':
        n_succeeded += 1
    elif j.status.lower() == 'running':
        n_running += 1
    elif j.status.lower() == 'accepted':
        n_accepted += 1
    else:
        print(j.status)

print("failed: ", n_failed)
print("accepted: ", n_accepted)
print("running: ", n_running)
print("succeeded: ", n_succeeded)
print("except: ", n_except)

failed:  0
accepted:  0
running:  0
succeeded:  0
except:  2


In [20]:
print(jobs[0])

{'job_id': '2ddea507-35c8-44d3-a54f-526467c51d11', 'status': 'success', 'machine_type': None, 'architecture': None, 'machine_memory_size': None, 'directory_size': None, 'operating_system': None, 'job_start_time': None, 'job_end_time': None, 'job_duration_seconds': None, 'cpu_usage': None, 'cache_usage': None, 'mem_usage': None, 'max_mem_usage': None, 'swap_usage': None, 'read_io_stats': None, 'write_io_stats': None, 'sync_io_stats': None, 'async_io_stats': None, 'total_io_stats': None, 'error_details': None, 'response_code': 200, 'outputs': []}


## Test Run in the ADE

In [3]:
# To run from the ADE, update the conda environment:
# (For DPS, this step is handled during algorithm registration.)
!conda env update -n base --file environment.yaml

Retrieving notices: ...working... done
Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 23.7.4
    latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



certifi-2023.11.17   | 155 KB    |                                       |   0% 
sardem-0.11.3        | 31 KB     |                                       |   0% 

ca-certificates-2023 | 151 KB    |                                       |   0% 


openssl-3.2.0        | 2.7 MB    |                                       |   0% 

ca-certificates-2023 | 151 KB    | ###########8                          |  32% 

ca-certificates-2023 | 151 KB    | ##################################### | 100% 


certifi-2023.11.17   | 155 KB    | ###8                                  |  10% 


certifi-2023.11.17   | 155 KB    | ##############################5       |  82% 
sardem-0.11.3        | 31 KB  

In [15]:
# WARNING - only use for testing! Not DPS!
import os
from time import time

# make the output directory, if it does not already exist
# os.makedirs("output")

# Build a DEM
start = time()

# Do a test run via CLI.
# DPS runs via CLI. So, use this method to ensure the
# inputs are being passed correctly to the algorithm.
# !./run.sh -118.06817 34.22169 -118.05801 34.22822  # small bbox; should take 7-10 seconds.
!./run.sh {bbox}

print(f"Time to make DEM: {time()-start} seconds")


[01/26 13:43:16] [INFO dem.py] Bounds: -118.06817 34.22169 -118.05801 34.22822
[01/26 13:43:16] [INFO cop_dem.py] Creating output/dem.tif
[01/26 13:43:16] [INFO cop_dem.py] Fetching remote tiles...
[01/26 13:43:16] [INFO cop_dem.py] Running GDAL command:
[01/26 13:43:16] [INFO cop_dem.py] gdalwarp /vsicurl/https://raw.githubusercontent.com/scottstanie/sardem/master/sardem/data/cop_global.vrt output/dem.tif -of GTiff -ot Int16 -te -118.068169999999995 34.2216900000000024 -118.058009999999996 34.2282200000000003 -tr 0.000277777777777777778 0.000277777777777777778 -s_srs "epsg:4326+3855" -t_srs "epsg:4326" -wo NUM_THREADS=4 -r nearest -wm 5000 -multi
Creating output file that is 37P x 24L.
0...10...20...30...40...50...60...70...80...90...100 - done.
Time to make DEM: 8.959450721740723 seconds
